**Install Dependencies:**

In [1]:
!pip -q install git+https://github.com/openai/swarm.git
!pip install openai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 26.4 MB/s eta 0:00:00


**openai client calling grow model**

In [22]:
import openai
from google.colab import userdata
#model = "meta/llama-3.1-405b-instruct"
#model = "llama-3.1-70b-versatile"
model = "llama-3.2-90b-text-preview"

model1 = "llama3-groq-70b-8192-tool-use-preview"

llm_client = openai.OpenAI(
  base_url="https://api.groq.com/openai/v1",
  api_key=userdata.get('GROQ_API_KEY'),
)



**Creating agents and functions:**

In [26]:
from swarm import Agent


def process_refund(item_id, reason="NOT SPECIFIED"):
    """Refund an item. Refund an item. Make sure you have the item_id of the form item_... Ask for user confirmation before processing the refund."""
    print(f"[mock] Refunding item {item_id} because {reason}...")
    return "Success!"


def apply_discount():
    """Apply a discount to the user's cart."""
    print("[mock] Applying discount...")
    return "Applied discount of 11%"


triage_agent = Agent(
    name="Triage Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
    model=model1,
    tool_choice="auto",
    max_turns=2
)
sales_agent = Agent(
    name="Sales Agent",
    instructions="Be super enthusiastic about selling bees.",
    model=model,
    tool_choice="auto",
    max_turns=2
)
refunds_agent = Agent(
    name="Refunds Agent",
    instructions="Help the user with a refund. If the reason is that it was too expensive, offer the user a refund code. If they insist, then process the refund.",
    functions=[process_refund, apply_discount],
    model=model,
    tool_choice="auto",
    max_turns=2
)


def transfer_back_to_triage(**kwargs):
    """Call this function if a user is asking about a topic that is not handled by the current agent."""
    return triage_agent


def transfer_to_sales(**kwargs):
    return sales_agent


def transfer_to_refunds(**kwargs):
    return refunds_agent


triage_agent.functions = [transfer_to_sales, transfer_to_refunds]
sales_agent.functions.append(transfer_back_to_triage)
refunds_agent.functions.append(transfer_back_to_triage)

In [27]:

refunds_agent

Agent(name='Refunds Agent', model='llama-3.2-90b-text-preview', instructions='Help the user with a refund. If the reason is that it was too expensive, offer the user a refund code. If they insist, then process the refund.', functions=[<function process_refund at 0x7bd201fcd5a0>, <function apply_discount at 0x7bd201fcd630>, <function transfer_back_to_triage at 0x7bd201fcd6c0>], tool_choice='auto', parallel_tool_calls=True)

In [28]:
triage_agent

Agent(name='Triage Agent', model='llama3-groq-70b-8192-tool-use-preview', instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.", functions=[<function transfer_to_sales at 0x7bd201fcd750>, <function transfer_to_refunds at 0x7bd201fcd7e0>], tool_choice='auto', parallel_tool_calls=True)

In [29]:
sales_agent

Agent(name='Sales Agent', model='llama-3.2-90b-text-preview', instructions='Be super enthusiastic about selling bees.', functions=[<function transfer_back_to_triage at 0x7bd201fcd6c0>], tool_choice='auto', parallel_tool_calls=True)

**printing reponse of agents**

In [19]:
import json

from swarm import Swarm


def process_and_print_streaming_response(response):
    content = ""
    last_sender = ""

    for chunk in response:
        if "sender" in chunk:
            last_sender = chunk["sender"]

        if "content" in chunk and chunk["content"] is not None:
            if not content and last_sender:
                print(f"\033[94m{last_sender}:\033[0m", end=" ", flush=True)
                last_sender = ""
            print(chunk["content"], end="", flush=True)
            content += chunk["content"]

        if "tool_calls" in chunk and chunk["tool_calls"] is not None:
            for tool_call in chunk["tool_calls"]:
                f = tool_call["function"]
                name = f["name"]
                if not name:
                    continue
                print(f"\033[94m{last_sender}: \033[95m{name}\033[0m()")

        if "delim" in chunk and chunk["delim"] == "end" and content:
            print()  # End of response message
            content = ""

        if "response" in chunk:
            return chunk["response"]


def pretty_print_messages(messages) -> None:
    for message in messages:
        if message["role"] != "assistant":
            continue

        # print agent name in blue
        print(f"\033[94m{message['sender']}\033[0m:", end=" ")

        # print response, if any
        if message["content"]:
            print(message["content"])

        # print tool calls in purple, if any
        tool_calls = message.get("tool_calls") or []
        if len(tool_calls) > 1:
            print()
        for tool_call in tool_calls:
            f = tool_call["function"]
            name, args = f["name"], f["arguments"]
            arg_str = json.dumps(json.loads(args)).replace(":", "=")
            print(f"\033[95m{name}\033[0m({arg_str[1:-1]})")

**creating run loop**

In [30]:
from uuid import uuid4  # to generate unique IDs

def run_demo_loop(
    starting_agent, context_variables=None, stream=False, debug=False
) -> None:
    client = Swarm(client=llm_client)
    print("Starting Groq Swarm CLI 🐝")

    messages = []
    agent = starting_agent

    while True:

        user_input = input("\033[90mUser\033[0m: ")
        if "exit" in user_input:
            print("Good bye")
            break
        messages.append({"role": "user", "content": user_input})

        response = client.run(
            agent=agent,
            messages=messages,
            context_variables=context_variables or {},
            stream=stream,
            debug=debug,
            max_turns=2
        )

        if stream:
            response = process_and_print_streaming_response(response)
        else:
            # Add tool_call_id for tool messages
            for msg in response.messages:
                if msg.get("role") == "tool":
                    msg["tool_call_id"] = str(uuid4())  # add a unique ID for each tool call

            pretty_print_messages(response.messages)

        messages.extend(
            [{'role': msg['role'], 'content': msg['content'], 'tool_call_id': msg.get("tool_call_id")} if msg["role"] == "tool" else {'role': msg['role'], 'content': msg['content']}
             for msg in response.messages]
        )
        agent = response.agent


**running multi agents**

In [12]:
if __name__ == "__main__":
    run_demo_loop(triage_agent)

Starting Groq Swarm CLI 🐝
User: I'd like to request a refund for item_12345 because it was too expensive. Can you help me with that?
Triage Agent: Sure, I can assist you with that. Could you please provide me with your order number and the reason for the refund?
User: item_12345
Triage Agent: 
transfer_to_refunds("kwargs"= "item_12345")
transfer_to_sales("kwargs"= "item_12345")
User: Do you have any discounts available?
Sales Agent: We actually have a special promotion going on right now for our Beekeeper's Delight package. It includes a starter hive, protective clothing, and a queen bee - everything you need to start your beekeeping journey. And, we're offering a 10% discount on all packages purchased this month. Would you like more information on that?
User: Can you provide support with my account settings?
Sales Agent: transfer_back_to_triage("topic"= "account settings")
User: I'd like a refund for an item I bought.
Triage Agent: Could you please provide me with the order number and